<a href="https://colab.research.google.com/github/sebasruggero/curriculum/blob/main/MapCrimeApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install h3
!pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.4 MB/s eta 0:00:00


# Sección nueva

In [4]:
# Importa las bibliotecas necesarias
import pandas as pd
import h3
import folium

In [5]:
path = '/content/drive/Othercomputers/My Laptop/DataScience/Data/DataFrames/delitos_totales_2016_2021.csv'

In [6]:
df = pd.read_csv(path, sep= "\t" , on_bad_lines='warn')

<ipython-input-6-d74b52fcfda8>:1: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep= "\t" , on_bad_lines='warn')


In [39]:
print(df.dtypes)

Unnamed: 0                      int64
fecha                  datetime64[ns]
franja_horaria                 object
tipo_delito                    object
subtipo_delito                 object
cantidad_registrada           float64
comuna                        float64
barrio                         object
lat                           float64
long                          float64
uso_armas                      object
h3_index                       object
dtype: object


In [20]:
# Función para limpiar valores no numéricos
def clean_numeric(value):
    try:
        return float(value)
    except ValueError:
        return None  # Otra opción podría ser reemplazar con un valor predeterminado

# Aplicar la función de limpieza a las columnas
df['lat'] = df['lat'].apply(clean_numeric)
df['long'] = df['long'].apply(clean_numeric)

# Eliminar filas con valores no numéricos si es necesario
df = df.dropna(subset=['lat', 'long'])

# Ahora, 'lat' y 'long' deberían contener números de punto flotante
print(df)

        Unnamed: 0      fecha franja_horaria            tipo_delito  \
0                0 2016-01-01             13   Robo (con violencia)   
1                1 2016-01-01             22   Robo (con violencia)   
2                2 2016-01-01              1               Lesiones   
3                3 2016-01-01             19  Hurto (sin violencia)   
4                4 2016-01-01              4   Robo (con violencia)   
...            ...        ...            ...                    ...   
560084       66720 2020-12-31              4   Robo (con violencia)   
560085       66721 2020-12-31             16   Robo (con violencia)   
560086       66722 2020-12-31             13   Robo (con violencia)   
560087       66723 2020-12-31              4   Robo (con violencia)   
560088       66724 2020-12-31              9   Robo (con violencia)   

        subtipo_delito  cantidad_registrada  comuna             barrio  \
0                  NaN                  1.0     9.0  Parque Avellaneda   

In [21]:
data = df

In [31]:
! pip install ipyleaflet ipywidgets streamlit

In [38]:
from h3 import h3
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from ipywidgets import interact, Dropdown


# Define the resolution of the H3 hexagons (adjust as needed)
resolution = 8

# Convert latitude and longitude coordinates to H3 indices
data['h3_index'] = data.apply(lambda row: h3.geo_to_h3(row['lat'], row['long'], resolution), axis=1)

# Create a colormap for the density of cases
cmap = cm.YlOrRd

# Create an interactive map with Folium
m = folium.Map(location=[-34.6131500, -58.3772300], zoom_start=11)

# Define a function to update the map based on the selected tipo_delito
def update_map(selected_tipo_delito):
    m = folium.Map(location=[-34.6131500, -58.3772300], zoom_start=11)

    # Filter data by the selected tipo_delito
    filtered_data = data[data['tipo_delito'] == selected_tipo_delito]

    # Count the density of cases per H3 index for the selected tipo_delito
    density_data = filtered_data['h3_index'].value_counts().reset_index()
    density_data.columns = ['h3_index', 'crime_count']

    # Add the density hexagons to the map for the selected tipo_delito
    for index, row in density_data.iterrows():
        h3_index = row['h3_index']
        crime_count = row['crime_count']
        lat, long = h3.h3_to_geo(h3_index)

        # Get the hexagon color based on density
        color = mcolors.rgb2hex(cmap(crime_count / density_data['crime_count'].max()))

        folium.Polygon(locations=h3.h3_to_geo_boundary(h3_index),
                       color='white',
                       weight=1,
                       fill=True,
                       fill_color=color,
                       fill_opacity=0.8,
                       popup=f'Cases: {crime_count}').add_to(m)


    return m

# Get unique tipo_delito values
tipo_delito_options = list(data['tipo_delito'].unique())

# Create a dropdown widget for selecting tipo_delito
tipo_delito_dropdown = Dropdown(options=tipo_delito_options, description='Tipo de Delito')

# Create an interactive map with the initial tipo_delito
interact(update_map, selected_tipo_delito=tipo_delito_dropdown)



interactive(children=(Dropdown(description='Tipo de Delito', options=('Robo (con violencia)', 'Lesiones', 'Hur…

<function __main__.update_map(selected_tipo_delito)>